In [1]:
import tensorflow as tf
assert(tf.VERSION >= '1.0.0')
import numpy as np
import matplotlib as mplib
import matplotlib.pyplot as plt
import json
import os
from six.moves import cPickle as pickle

%matplotlib inline

In [2]:
pickle_file = 'dataset_labels100by100.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  #valid_dataset = save['valid_dataset']
  #valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  #print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)


#dataset = np.reshape(train_dataset, [train_dataset.shape[0], 50,50,1])
#labels = (np.arange(1,13) == train_labels[:,None]).astype(np.float32)


('Training set', (1000, 100, 100), (1000,))
('Test set', (250, 100, 100), (250,))


In [3]:
dataset.shape
plt.imshow(dataset[11,:,:,0])
labels[11,:]

NameError: name 'dataset' is not defined

In [3]:
def getParameters():
    # Network parameters (available at the command line)
    flags = tf.app.flags
    flags.DEFINE_integer('instance', 0, "Instance index")
    flags.DEFINE_integer('training_epochs', 15, "Number of training epochs")
    flags.DEFINE_float('learning_rate', 0.001, "Learning rate")
    flags.DEFINE_integer('batch_size', 100, "batch size for learning")
    flags.DEFINE_integer('n_conv_1', 32, "number of filters at first convolutional hidden level")
    flags.DEFINE_integer('n_conv_2', 64, "number of filters at second convolutional hidden level")
    flags.DEFINE_integer('n_pool_1', 2, "size of pooling nodes at first pooling hidden level")
    flags.DEFINE_integer('n_pool_2', 2, "size of pooling nodes at second pooling hidden level")
    flags.DEFINE_integer('n_rec', 20, "number of nodes at first reconstruction hidden level")
    flags.DEFINE_float('dropout', 0.4, "Dropout regularization probability")
    flags.DEFINE_string('log_dir', '/tmp/logs', "Directory to write log files in")
    return flags.FLAGS

In [4]:

IMG_DIMS = [100, 100, 1]  #images are 28x28, single channel
KERNEL_SIZE = [7,7] #convolutional filter kernel size
N_CATS = 10
N_INPUT = 100*100*1

def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:,:,:]
  shuffled_labels = labels[permutation,:]
  return shuffled_dataset, shuffled_labels


def buildGraph(flags):
    """
    Bulds the graph
    """
    
    x = tf.placeholder("float", [None, N_INPUT], name="input")
    y = tf.placeholder("float", [None, N_CATS], name = "label")
    isTraining = tf.placeholder("bool", [], name="is_training");
    # conv layers
    x = tf.reshape(x, [flags.batch_size, IMG_DIMS[0], IMG_DIMS[1], IMG_DIMS[2]])
    #conv1 = tf.nn.relu(tf.nn.conv2d(x, weights['filter1'], strides = [1,1,1,1], padding = "SAME" ))
    
    conv1 = tf.layers.conv2d(
            inputs=x,
            filters=flags.n_conv_1,
            kernel_size=KERNEL_SIZE,
            padding="SAME",
            activation=tf.nn.relu,
            name='v1')
    
    pool1 = tf.layers.max_pooling2d(
            inputs=conv1,
            pool_size=flags.n_pool_1,
            strides=2)
    
    dropout = tf.layers.dropout(
            inputs = pool1, 
            rate = flags.dropout, 
            training = isTraining)
    
    #conv2 = tf.nn.relu(tf.nn.conv2d(pool1, weights['filter2'], strides = [1,1,1,1], padding = "SAME" ))
    conv2 = tf.layers.conv2d(
            inputs=dropout,
            filters=flags.n_conv_2,
            kernel_size=KERNEL_SIZE,
            padding="SAME",
            activation=tf.nn.relu,
            name='v2')
    
    pool2 = tf.layers.max_pooling2d(
            inputs=conv2, 
            pool_size=flags.n_pool_2,
            strides=2)

    #fully connected dense layer
    flat2 = tf.contrib.layers.flatten(pool2)
    flat1 = tf.contrib.layers.flatten(pool1)
    conc_out = tf.concat([flat1,flat2],1)
    
    class1 = tf.layers.dense(
              inputs = conc_out,   
              units = flags.n_rec,
              activation = tf.nn.relu
             )
    
    dropout = tf.layers.dropout(
            inputs = class1, 
            rate = flags.dropout, 
            training = isTraining)
    
    y_pred = tf.layers.dense(
            inputs = dropout,
            units = N_CATS
            )

    # Learning criteria
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y))
    optimizer = tf.train.RMSPropOptimizer(flags.learning_rate).minimize(cost)
    
    # Evaluate model
    correct_pred = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
    graph = {'input': x, 
             'output': y_pred, 
             'cost': cost, 
             'optimizer': optimizer,
             'accuracy': accuracy,
             'labels': y
            }
    
    return (graph)


In [5]:
def trainclassifier(tfSession, dataset, graph, flags, saver, labels, test_dataset, test_labels):
    """
    Trains a built graph. The variables should be previously initialized
    """
    # Define loss and optimizer, minimize the squared error
    x = graph['input']
    cost = graph['cost']
    optimizer = graph['optimizer']
    accuracy = graph['accuracy']
    y = graph['labels']
    
    dataset = np.reshape(dataset, [dataset.shape[0], IMG_DIMS[0], IMG_DIMS[1], IMG_DIMS[2]])
    labels = (np.arange(1,N_CATS+1) == labels[:,None]).astype(np.float32)
    
    isTraining = tfSession.graph.get_tensor_by_name("is_training:0")
    # Training cycle
    nBatches = int(dataset.shape[0]/flags.batch_size)
    c = np.inf
    for epoch in range(1, 1+flags.training_epochs):
        # Loop over all batches
        start = 0
        end = flags.batch_size
        dataset, labels = randomize(dataset, labels)
        for i in range(nBatches):
            step = (epoch - 1) * nBatches + i
            # get next batch of images. 
            xBatch = dataset[start:end,:,:,:]
            yBatch = labels[start:end, :]
            
            # Run optimization and cost op, get accuracy
            _, c, acc = tfSession.run(
                    [optimizer, cost, accuracy], 
                    feed_dict={x: xBatch, y: yBatch, isTraining: True })
            savepath = saver.save(tfSession, 'Supervised_100by100')
            start = start + flags.batch_size
            end = end + flags.batch_size
            print "Epoch: {0:3d}, step = {1:3d} cost = {2:.6f} accuracy = {3:.6f} variables saved to {4}".format(epoch, step, c, acc, savepath )
        testModel(tfSession, test_dataset, graph, flags, test_labels)    
    return acc

In [6]:
def testModel(tfSession, dataset, graph, flags, labels):
    """
    Tests the models by checking accuracy on testset
    """
    # Applying encoder and decoder over test set
    x = graph['input']
    acc = graph['accuracy']
    y = graph['labels']
    
    v1filters = tfSession.graph.get_collection('trainable_variables', 'v1')[0]
    v2filters = tfSession.graph.get_collection('trainable_variables', 'v2')[0]
    
    isTraining = tfSession.graph.get_tensor_by_name('is_training:0')
    dataset = np.reshape(dataset, [dataset.shape[0], IMG_DIMS[0], IMG_DIMS[1], IMG_DIMS[2]])
    labels = (np.arange(1,N_CATS+1) == labels[:,None]).astype(np.float32)
    
    nBatches = int(dataset.shape[0]/flags.batch_size)
    start = 0
    end = flags.batch_size
    
    for i in range(nBatches):
        output, accuracy, v1imgs, v2imgs, c = tfSession.run(
            [graph['output'], acc, v1filters, v2filters, graph['cost']], 
            feed_dict={x: dataset[start:end,:,:,:], y: labels[start:end,:], isTraining: False  })
        # Display accuracy for each
        print "Batch: {0:3d}, cost : {1:.6f}, accuracy = {2:.6f}".format(i, c, accuracy)
        start = start + flags.batch_size
        end = end + flags.batch_size
  
    
    #fV1, aV1 = mplib.pyplot.subplots(8, flags.n_conv_1/8)
    #assert( v1imgs.shape[-1] == flags.n_conv_1 )
    #for i in range(flags.n_conv_1):
     #   aV1[i%8][i/8].imshow(v1imgs[:,:,0,i], cmap='gray')
#        aV1[i%8][i/8].imshow(v1imgs[:,:,0,i], cmap='gray', interpolation='nearest')
    #fV1.show()
    
    #fV2, aV2 = mplib.pyplot.subplots(8, 8)
    #for i in range(64):
    #    aV2[i%8][i/8].imshow(v2imgs[:,:,0,i], cmap='gray')
    #fV2.show()

In [7]:
flags = getParameters()


#Build graph
graph = buildGraph(flags)

sess = tf.InteractiveSession()

saver = tf.train.Saver(max_to_keep = 1)
saver2 = tf.train.Saver({"v1": sess.graph.get_collection('trainable_variables', 'v1')[0], 
                     "v2": sess.graph.get_collection('trainable_variables', 'v2')[0]})

# Initialize the variables
sess.run(tf.global_variables_initializer())
#print sess.graph.get_collection('trainable_variables', 'v1')[0].eval()
#saver2.restore(sess, '/Users/raghu/DeepCat/model1_autoenc')
#print sess.graph.get_collection('trainable_variables', 'v1')[0].eval()

testModel(sess, test_dataset, graph, flags, test_labels)
cost = -1
for majorEpoch in range(10):
    print 'Major Epoch {0}:'.format(majorEpoch+1)
    #Train model    
    accuracy = trainclassifier(sess, train_dataset, graph, flags, saver, train_labels, test_dataset, test_labels)
    #Test model
    testModel(sess, test_dataset, graph, flags, test_labels)
    
print 'Optimization Finished!'
testModel(sess, test_dataset, graph, flags, test_labels)

Batch:   0, cost : 2.328967, accuracy = 0.140000
Batch:   1, cost : 2.320223, accuracy = 0.110000
Major Epoch 1:
Epoch:   1, step =   0 cost = 2.337038 accuracy = 0.100000 variables saved to Supervised_100by100
Epoch:   1, step =   1 cost = 2.301084 accuracy = 0.140000 variables saved to Supervised_100by100
Epoch:   1, step =   2 cost = 2.289130 accuracy = 0.140000 variables saved to Supervised_100by100
Epoch:   1, step =   3 cost = 2.261693 accuracy = 0.130000 variables saved to Supervised_100by100
Epoch:   1, step =   4 cost = 2.315495 accuracy = 0.110000 variables saved to Supervised_100by100
Epoch:   1, step =   5 cost = 2.285810 accuracy = 0.110000 variables saved to Supervised_100by100
Epoch:   1, step =   6 cost = 2.296791 accuracy = 0.140000 variables saved to Supervised_100by100
Epoch:   1, step =   7 cost = 2.300026 accuracy = 0.080000 variables saved to Supervised_100by100
Epoch:   1, step =   8 cost = 2.277757 accuracy = 0.180000 variables saved to Supervised_100by100
Epoch